<a href="https://colab.research.google.com/github/seawavve/PeekABook/blob/main/model/custom_trans.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

setup

In [1]:
!pip3 install --quiet tensorflow
!pip3 install --quiet tensorflow_text

     |████████████████████████████████| 3.4MB 19.6MB/s 


In [2]:
import numpy as np
import pandas as pd
%matplotlib inline  
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import seaborn as sns
from sklearn.metrics import pairwise
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text  # Imports TF ops for preprocessing.

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd
import tensorflow_hub as hub

In [3]:
class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = keras.Sequential(
            [layers.Dense(ff_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)


In [4]:
class TokenAndPositionEmbedding(layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = layers.Embedding(input_dim=maxlen, output_dim=embed_dim)

    def call(self, x):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions


In [5]:
#@title Configure the model { run: "auto" }
BERT_MODEL = "https://tfhub.dev/google/experts/bert/wiki_books/sst2/2" # @param {type: "string"} ["https://tfhub.dev/google/experts/bert/wiki_books/2", "https://tfhub.dev/google/experts/bert/wiki_books/mnli/2", "https://tfhub.dev/google/experts/bert/wiki_books/qnli/2", "https://tfhub.dev/google/experts/bert/wiki_books/qqp/2", "https://tfhub.dev/google/experts/bert/wiki_books/squad2/2", "https://tfhub.dev/google/experts/bert/wiki_books/sst2/2",  "https://tfhub.dev/google/experts/bert/pubmed/2", "https://tfhub.dev/google/experts/bert/pubmed/squad2/2"]
# Preprocessing must match the model, but all the above use the same.
PREPROCESS_MODEL = "https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3"

prepare dataset

In [6]:
vocab_size = 20000  # Only consider the top 20k words
maxlen = 200  # Only consider the first 200 words of each x_data
data=pd.read_csv('./Peterpan_emo3.csv')
data=data.dropna(how='any')
print(f'Dimensions: {data.shape}')
data

Dimensions: (1697, 4)


,Unnamed: 0,sentence,posNeg,posNeg3
0,0,Chapter 1 PETER BREAKS THROUGH,0.000000,1.0
1,1,"All children, except one, grow up. They soon k...",0.311111,2.0
2,2,Of course they lived at 14 [their house number...,0.279545,2.0
3,3,The way Mr. Darling won her was this: the many...,0.483333,2.0
4,4,Mr. Darling used to boast to Wendy that her mo...,0.148889,1.0
...,...,...,...,...
1692,1692,"“If only I could go with you,” Wendy sighed.",0.000000,1.0
1693,1693,"“You see you can't fly,” said Jane.",0.800000,2.0
1694,1694,Of course in the end Wendy let them fly away t...,0.183333,2.0
1695,1695,"As you look at Wendy, you may see her hair bec...",0.097396,1.0


In [11]:
sentences=data['sentence'].tolist()
Y_data= data['posNeg3'].tolist()
preprocess = hub.load(PREPROCESS_MODEL)
bert = hub.load(BERT_MODEL)
inputs = preprocess(sentences[:500])
bert_results = bert(inputs)
print(bert_results)

{'encoder_outputs': [<tf.Tensor: shape=(500, 128, 768), dtype=float32, numpy=
array([[[ 0.0011499 , -0.02532766,  0.0496476 , ...,  0.24924558,
         -0.03188992,  0.00629036],
        [ 0.5109046 , -0.61041427,  0.6048647 , ..., -0.20907308,
         -0.40722883, -0.27833563],
        [ 0.11850551, -0.34156585, -0.44050834, ..., -0.36718255,
          0.03220936,  0.10791692],
        ...,
        [-0.17884171, -0.10139102,  0.364949  , ...,  0.34813005,
         -0.18461484, -0.19746351],
        [-0.09677637, -0.08950923,  0.47329453, ...,  0.4158615 ,
         -0.21884018, -0.23421523],
        [-0.10471042, -0.16743065,  0.4955452 , ...,  0.43011165,
         -0.18079133, -0.2474733 ]],

       [[ 0.02463844,  0.03560804,  0.04623216, ...,  0.2842862 ,
         -0.00575601, -0.03268295],
        [-0.12171297, -0.00797039, -0.35404497, ..., -0.62068224,
         -0.12919202,  0.60665375],
        [-0.9193618 , -0.15029505, -0.29354283, ..., -0.22051197,
          0.7163553 ,  0.

In [8]:
print(f'Pooled Outputs Shape:{bert_results["pooled_output"].shape}')
print(f'Pooled Outputs Values:{bert_results["pooled_output"][0, :12]}')
print(f'Sequence Outputs Shape:{bert_results["sequence_output"].shape}')
print(f'Sequence Outputs Values:{bert_results["sequence_output"][0, :12]}')

Pooled Outputs Shape:(500, 768)
Pooled Outputs Values:[ 0.45421395  0.2949192   0.79641217  0.60480106  0.3280845   0.03076395
 -0.28662452 -0.61797523  0.7500393   0.03723879  0.5965795  -0.54139286]
Sequence Outputs Shape:(500, 128, 768)
Sequence Outputs Values:[[ 0.48999685  0.30394557  1.0887244  ...  1.848758    0.16467124
  -0.325687  ]
 [ 0.6065986  -0.47776282  0.2609226  ...  2.3007226  -0.37581712
  -0.05384592]
 [ 0.17835021  0.0432563   0.5358286  ...  1.7011096  -0.8225711
  -0.31128657]
 ...
 [ 0.44960898  0.07259469  0.03648981 ...  0.00718453 -1.6866527
  -0.32370418]
 [ 0.46865958 -0.03646746  0.08884454 ...  0.01199529 -1.8194145
  -0.43946   ]
 [ 0.44177595 -0.06479785  0.04875329 ...  0.06126611 -1.7820753
  -0.42757037]]


Create classifier model using transformer layer

In [13]:
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense, GRU, Embedding

# model = Sequential()
# model.add(Embedding(vocab_size, 100))
# model.add(GRU(128))
# model.add(Dense(1, activation='sigmoid'))
# model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
# history = model.fit(x_train, y_train, epochs=15, batch_size=60, validation_split=0.2)

In [16]:
embed_dim = 768  # Embedding size for each token
num_heads = 3  # Number of attention heads
ff_dim = 32  # Hidden layer size in feed forward network inside transformer

x = layers.Input(shape=(128,768))
# embedding_layer = TokenAndPositionEmbedding(maxlen, vocab_size, embed_dim)
# x = embedding_layer(x)
transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim)
x = transformer_block(x)
x = layers.GlobalAveragePooling1D()(x)
x = layers.Dropout(0.1)(x)
x = layers.Dense(20, activation="relu")(x)
x = layers.Dropout(0.1)(x)
outputs = layers.Dense(3, activation="softmax")(x)  #출력층은 3개 여야지

model = keras.Model(inputs=inputs, outputs=outputs)


ValueError: ignored

In [ ]:
model.compile("adam", "sparse_categorical_crossentropy", metrics=["accuracy"])
history = model.fit(
    x_train, y_train, batch_size=32, epochs=2, validation_data=(x_val, y_val)
)

NameError: ignored

In [ ]:
import matplotlib.pyplot as plt
# Fit the model
results = history
# list all data in history
print(results.history.keys())
# summarize history for accuracy
plt.plot(results.history['accuracy'])
plt.plot(results.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(results.history['loss'])
plt.plot(results.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()